Now, we are going to do similar thing using pipeline

In [1]:
#@ Importing the libraries
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
#@ Getting the datasets
data = pd.read_csv('titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## plan for creating the pipeline

Here, we have missing values in the two columns i.e. Age & Embarked. So, first step for the pipeline is to:
- `impute` the missing values using 1 column transformer
- then we will send it to 2nd column transformer which task is to do `OneHotEncoding` for two columns i.e. `Sex` and `Embarked`
- Then, from the above output we will do `scaling`
- And, we will do `feature selection` for getting top features
- We will train our model using `DecisionTreeClassifier` which will give the output

In [4]:
# dropping unnecessary columns
data.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [5]:
# doing train/test/split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns = ['Survived']),
                                                   data['Survived'],
                                                   test_size = 0.2,
                                                   random_state = 42)

In [6]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [7]:
y_train.sample(5)

52     1
430    1
151    1
102    0
214    0
Name: Survived, dtype: int64

In [8]:
#@ step:1 ------> ImputationTransformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),         # creating simpleimputer in numpy array
    ('impute_embarked', SimpleImputer(strategy = 'most_frequent'), [6])      # simpleimputer for embarked
], remainder = 'passthrough')

In [9]:
#@ step:2 ------> OneHotEncoding
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse = False, handle_unknown = 'ignore'),[1,6])
], remainder = 'passthrough')

In [10]:
#@ step:3 ------> Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))       # we are doing feature selection so, we will be using MinMaxScaler
])

In [11]:
#@ step:4 ------> FeatureSelection
trf4 = SelectKBest(score_func = chi2, k = 8)

In [12]:
#@ step:5 ------> Training the mode
trf5 = DecisionTreeClassifier()

## Create Pipeline
Now, we are ready for creating the pipeline 

In [13]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

## Pipeline V/S make_pipeline

Pipeline requires naming of steps, make_pipeline does not

(Same applies to ColumnTransformer v/s make_column_transformer)

In [14]:
#@ Alternative Syntx
pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [15]:
#@ now, training the data
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                             

## Exploring the pipeline

In [16]:
#@ Display the pipeline
from sklearn import set_config
set_config(display = 'diagram')

In [17]:
#@ getting the steps
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x0000020E37F70430>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [18]:
#@ Predicting the data
y_predict = pipe.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_predict)

0.6256983240223464

## Cross Validation using Pipeline

In [20]:
#@ cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv = 5, scoring = 'accuracy').mean()

0.6391214419383433

## GridSearch using Pipeline

In [21]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [22]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

ValueError: Invalid parameter trf5 for estimator Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scal

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

## Exporting the Pipeline

In [23]:
#@ Export 
import pickle 
pickle.dump(pipe, open('pipe.pkl', 'wb'))